In [3]:
import org.apache.spark.sql._
import org.apache.spark.ml._
import java.io._
import play.api.libs.json._
import org.apache.spark.sql.Row
import org.apache.spark.sql.functions._

In [4]:
val vehicleUsers = spark.read.json("/labs/laba02/autousers.json").toDF()

Waiting for a Spark session to start...

vehicleUsers = [autousers: array<string>]


[autousers: array<string>]

In [5]:
vehicleUsers.printSchema()

root
 |-- autousers: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [6]:
vehicleUsers.show(numRows = 5, truncate = 100, vertical=true)

-RECORD 0---------------------------------------------------------------------------------------------------------
 autousers | [100341861572, 100473724387, 100528753939, 101182896723, 101818209438, 101886368239, 102535398052... 



In [7]:
val logs = spark.read.option("delimiter", "\t").textFile("/labs/laba02/logs")

logs = [value: string]


[value: string]

In [8]:
logs.show(numRows = 5, truncate = 100, vertical=true)

-RECORD 0-----------------------------------------------------------------------------------------------------
 value | 258131083251	1423724291.637	http%3A%2F%2Fwww.zakon.kz%2F                                             
-RECORD 1-----------------------------------------------------------------------------------------------------
 value | 258131083251	1423724290.637                                                                          
-RECORD 2-----------------------------------------------------------------------------------------------------
 value | 182984926014	1423724291.666	http%3A%2F%2Fwww.bmwclub.ru%2Fvb%2Fshowthread.php%3Ft%3D875590%26page... 
-RECORD 3-----------------------------------------------------------------------------------------------------
 value | 182984926014	1423724290.666	http%3A%2F%2Fwww.bmwclub.ru%2Fvb%2Fshowthread.php%3Ft%3D875590%26page... 
-RECORD 4-----------------------------------------------------------------------------------------------------
 

In [9]:
logs.show(10)

+--------------------+
|               value|
+--------------------+
|258131083251	1423...|
|258131083251	1423...|
|182984926014	1423...|
|182984926014	1423...|
|289380960531	1423...|
|289380960531	1423...|
|85356233460	14237...|
|204109491902	1423...|
|204109491902	1423...|
|302580370974	1423...|
+--------------------+
only showing top 10 rows



In [10]:
// val logs = logs.map(line => (line.split("\t")(1).split(" ").flatMap(word => word), line.split("\t")(0))).map(word => (word, 1)).reduceByKey(_ + _)


Name: Syntax Error.
Message: 
StackTrace: 

In [10]:
logs.show(3, truncate=false)

+-----------------------------------------------------------------------------------------------------+
|value                                                                                                |
+-----------------------------------------------------------------------------------------------------+
|258131083251	1423724291.637	http%3A%2F%2Fwww.zakon.kz%2F                                             |
|258131083251	1423724290.637                                                                          |
|182984926014	1423724291.666	http%3A%2F%2Fwww.bmwclub.ru%2Fvb%2Fshowthread.php%3Ft%3D875590%26page%3D5|
+-----------------------------------------------------------------------------------------------------+
only showing top 3 rows



In [16]:
val col_ = split(col("value"), "\t").alias("value")
val logs_splited = logs.select(col_)
logs_splited.show(3, truncate=false)

+---------------------------------------------------------------------------------------------------------+
|value                                                                                                    |
+---------------------------------------------------------------------------------------------------------+
|[258131083251, 1423724291.637, http%3A%2F%2Fwww.zakon.kz%2F]                                             |
|[258131083251, 1423724290.637]                                                                           |
|[182984926014, 1423724291.666, http%3A%2F%2Fwww.bmwclub.ru%2Fvb%2Fshowthread.php%3Ft%3D875590%26page%3D5]|
+---------------------------------------------------------------------------------------------------------+
only showing top 3 rows



col_ = split(value, 	) AS `value`
logs_splited = [value: array<string>]


[value: array<string>]

In [30]:
logs_splited.withColumn("UID", $"value"(0))
            .withColumn("Timestamp", $"value"(1))
            .withColumn("URL", $"value"(2))
            .select("UID","Timestamp", "URL")
            .show(10, truncate=false)

+------------+--------------+-------------------------------------------------------------------------+
|UID         |Timestamp     |URL                                                                      |
+------------+--------------+-------------------------------------------------------------------------+
|258131083251|1423724291.637|http%3A%2F%2Fwww.zakon.kz%2F                                             |
|258131083251|1423724290.637|null                                                                     |
|182984926014|1423724291.666|http%3A%2F%2Fwww.bmwclub.ru%2Fvb%2Fshowthread.php%3Ft%3D875590%26page%3D5|
|182984926014|1423724290.666|http%3A%2F%2Fwww.bmwclub.ru%2Fvb%2Fshowthread.php%3Ft%3D875590%26page%3D4|
|289380960531|1423724291.723|http%3A%2F%2Fwww.bmwclub.ru%2Fvb%2Fforumdisplay.php%3Ff%3D128            |
|289380960531|1423724290.723|http%3A%2F%2Fwww.bmwclub.ru%2Fvb%2F                                      |
|85356233460 |1423724290.923|http%3A//www.wmmail.ru/index.php%3F

In [18]:
logs_splited.select(posexplode($"value")).show()

+---+--------------------+
|pos|                 col|
+---+--------------------+
|  0|        258131083251|
|  1|      1423724291.637|
|  2|http%3A%2F%2Fwww....|
|  0|        258131083251|
|  1|      1423724290.637|
|  0|        182984926014|
|  1|      1423724291.666|
|  2|http%3A%2F%2Fwww....|
|  0|        182984926014|
|  1|      1423724290.666|
|  2|http%3A%2F%2Fwww....|
|  0|        289380960531|
|  1|      1423724291.723|
|  2|http%3A%2F%2Fwww....|
|  0|        289380960531|
|  1|      1423724290.723|
|  2|http%3A%2F%2Fwww....|
|  0|         85356233460|
|  1|      1423724290.923|
|  2|http%3A//www.wmma...|
+---+--------------------+
only showing top 20 rows



In [ ]:
DF.withColumn("IP Address",split(col("value")," - - ")(0)).
withColumn("temp1",split(col("value")," - - ")(1)).
withColumn("Time",concat(split(col("temp1")," ")(0),split(col("temp1")," ")(1))).
withColumn("col3",substring(split(col("temp1")," ")(2),2,3)).
withColumn("Col4",split(col("temp1")," ")(3)).
select(col("IP Address"),col("Time"),col("col3"),col("col4")).show

In [1]:
// Создаем новую колонку, разибая наши данные по \t
//val jsonStrings: Column = split(col("value"), "\t").alias("value")

// Используем функцию explode для того, чтобы развернуть массив мехом наружу и используем темную магию 
// для превращения DataFrame в Dataset[String]
//val splited: Dataset[String] = raw.select(explode(jsonStrings)).as[String]

Name: Syntax Error.
Message: 
StackTrace: 

In [15]:
logs.select(explode(split(col("value"), "\t")).alias("value")).show(numRows = 20, truncate = 100, vertical=true)

-RECORD 0--------------------------------------------------------------------------
 value | 258131083251                                                              
-RECORD 1--------------------------------------------------------------------------
 value | 1423724291.637                                                            
-RECORD 2--------------------------------------------------------------------------
 value | http%3A%2F%2Fwww.zakon.kz%2F                                              
-RECORD 3--------------------------------------------------------------------------
 value | 258131083251                                                              
-RECORD 4--------------------------------------------------------------------------
 value | 1423724290.637                                                            
-RECORD 5--------------------------------------------------------------------------
 value | 182984926014                                                       

In [16]:
logs.select(explode(split(col("value"), "\t").alias("value")) as (Seq("x", "y", "z"))).show()

Name: org.apache.spark.sql.AnalysisException
Message: The number of aliases supplied in the AS clause does not match the number of columns output by the UDTF expected 1 aliases but got x,y,z ;
StackTrace:   at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$class.failAnalysis(CheckAnalysis.scala:43)
  at org.apache.spark.sql.catalyst.analysis.Analyzer.failAnalysis(Analyzer.scala:95)
  at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveGenerate$.makeGeneratorOutput(Analyzer.scala:1931)
  at org.apache.spark.sql.catalyst.analysis.Analyzer$ExtractGenerator$$anonfun$apply$23$$anonfun$70.apply(Analyzer.scala:1878)
  at org.apache.spark.sql.catalyst.analysis.Analyzer$ExtractGenerator$$anonfun$apply$23$$anonfun$70.apply(Analyzer.scala:1866)
  at scala.collection.TraversableLike$$anonfun$flatMap$1.apply(TraversableLike.scala:241)
  at scala.collection.TraversableLike$$anonfun$flatMap$1.apply(TraversableLike.scala:241)
  at scala.collection.mutable.ResizableArray$class.foreach(Resiz

In [2]:
logs.printSchema()

Name: Compile Error
Message: <console>:26: error: not found: value logs
       logs.printSchema()
       ^

StackTrace: 

In [18]:
vehicleUsers.select(explode($"autousers")).show()

+------------+
|         col|
+------------+
|100341861572|
|100473724387|
|100528753939|
|101182896723|
|101818209438|
|101886368239|
|102535398052|
|103624582558|
|103753741251|
|104091409587|
|104619031972|
|104636088851|
|104684233907|
|104979586580|
|105019547390|
|105036954964|
|105393450179|
|105489980463|
| 10565067157|
|105794662484|
+------------+
only showing top 20 rows



In [58]:
vehicleUsers.select($"autousers").show()

+--------------------+
|           autousers|
+--------------------+
|[100341861572, 10...|
+--------------------+



In [ ]:
import org.apache.spark.sql.functions._
appActiveTime.withColumn("data", explode($"data"))
       .select("data.*")
       .show(false)